## Librerias

In [1]:
#pip install Pulp 

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import os 
import pulp

## Lectura de datos

In [35]:
df_demanda = pd.read_excel('Dataton 2023 Etapa 1.xlsx', sheet_name="demand")
df_workers = pd.read_excel('Dataton 2023 Etapa 1.xlsx', sheet_name="workers")

In [39]:
# Convertimos la columna 'fecha_hora' esté en formato datetime
df_demanda['fecha_hora'] = pd.to_datetime(df_demanda['fecha_hora'])
# Define las variables binarias
empleados = df_workers['documento'].tolist()
franjas = range(len(df_demanda))
estados = ["Nada", "Trabaja", "PausaActiva", "Almuerzo"]
variables = [(empleado, franja, estado) for empleado in empleados for franja in franjas for estado in estados]
trabaja = pulp.LpVariable.dicts("Trabaja", variables, cat=pulp.LpBinary)

In [40]:
# Define la función objetivo para minimizar la diferencia entre la demanda y la capacidad
demandas = df_demanda['demanda'].tolist()
# Capacidad de empleados disponibles en una franja
capacidad = len(empleados)
# Crea el problema de minimización
prob = pulp.LpProblem("ProgramacionHoraria", pulp.LpMinimize)
prob += pulp.lpSum(demandas[i] - pulp.lpSum(trabaja[(e, i, 'Trabaja')] for e in empleados) for i in franjas)

In [41]:
# Restricción 1: Cada empleado debe cumplir 8 horas de trabajo (29 periodos de 15 minutos)
for empleado in empleados:
    prob += pulp.lpSum(trabaja[(empleado, i, 'Trabaja')] for i in franjas) == 29


In [43]:
# Restricción 2: Mínimo 4 periodos de 15 minutos y máximo 8 periodos de 15 minutos para pasar a "Pausa Activa"
for empleado in empleados:
    for franja in franjas:
        if franja >= 8:  # Asegúrate de que haya suficientes franjas anteriores
            # Agrega restricciones para "Pausa Activa"
            suma_trabajo = pulp.lpSum(trabaja[(empleado, i, 'Trabaja')] for i in range(franja - 8, franja))
            suma_pausa_activa = pulp.lpSum(trabaja[(empleado, i, 'PausaActiva')] for i in range(franja - 4, franja))
            prob += suma_trabajo - suma_pausa_activa <= 0
            prob += suma_pausa_activa - suma_trabajo <= 4  # Cambia 4 a tu máximo requerido


In [44]:
# Restricción 3: Tiempo de almuerzo de 6 periodos de 15 minutos
for empleado in empleados:
    for franja in franjas:
        if franja >= 6:  # Asegúrate de que haya suficientes franjas anteriores
            suma_almuerzo = pulp.lpSum(trabaja[(empleado, i, 'Almuerzo')] for i in range(franja - 6, franja))
            prob += suma_almuerzo <= 0  # No pueden haber más de 6 periodos de almuerzo seguidos


In [46]:
# Restricción 4: Al menos 1 empleado en el estado "Trabaja" en cada franja horaria
for franja in franjas:
    prob += pulp.lpSum(trabaja[(e, franja, 'Trabaja')] for e in empleados) >= 1


In [49]:
# Resuelve el problema
prob.solve()
# Crea un DataFrame para almacenar la solución
solucion_df = pd.DataFrame()
# Imprime la solución
if pulp.LpStatus[prob.status] == "Optimal":
    for empleado in empleados:
        for franja in franjas:
            for estado in estados:
                if pulp.value(trabaja[(empleado, franja, estado)]) == 1:
                    print(f"Empleado {empleado} está en estado {estado} en franja horaria {franja}")

# Exporta la solución 
solucion_df.to_csv("solucionprueba.csv", index=False)
